Alignement des entités nommées avec la base de données Wikidata et récupération des informations utiles à l'étude:
- Dates de naissance
- Description de l'entrée
- Genre du personnage

Outils: qwikidata, wptools

In [ ]:
# @title Installation des bibliothèques et connexion au Google Drive

"""Marina Giardinetti
marina.giardinetti@gmail.com
Google Colab
2024"""

!pip install qwikidata
import qwikidata
!pip install wptools
import wptools
from collections import Counter
import os
from json import JSONDecodeError
import pandas as pd
from google.colab import drive
import re
from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)
drive.mount('/content/drive/')

In [ ]:
# @title Création de nouveaux fichiers .xlsx contenant les informations récupérées
def get_infobox(donnees, itemid):
  birth, death, genre, description = ([] for i in range(4))
  page = wptools.page(wikibase=itemid, lang="fr")
  try:
    page.get_wikidata()
    page.get_parse()
    infobox = page.data['infobox']
    tout=page.data['wikidata']
    humain = page.data['what']
    genre=tout.get("sexe ou genre (P21)")
    nom = infobox.get("nom")
    nom= re.sub('{|}|<br/>|<br />\<br>|<small>|</small>|noble-\||souverain-\|', '', str(nom))
    description = page.data["description"]
    genre = re.sub(r'\(.*\)', '', str(genre))
    date_deces = infobox.get("date de décès")
    death = re.findall(r'\d{3,4}', str(date_deces))
    if len(death) >= 0:
      death = death[0]
    death = re.sub('[|]', '', str(death))
    date_naissance = infobox.get("date de naissance")
    birth = re.findall(r'\d{3,4}', str(date_naissance))
    birth = re.sub('[\'|\']', '', str(birth))
    donnees[nom]= {"Naissance" : birth, "Deces" : death, "Genre": genre, 'Description': description, 'URL': uri}
  except (AttributeError, LookupError):pass



def get_ID(title):
  title = title.lower().title()
  title = re.sub('(la )|(les )|(le )', '', title)
  print(title)
  sparql_query = ''.join(['SELECT ?item ?itemLabel WHERE {?item wdt:P31 wd:Q5.?item ?label "', title, '"@fr .SERVICE wikibase:label { bd:serviceParam wikibase:language "fr". }}'])
  try:
    res = return_sparql_query_results(sparql_query)
    print(res)
    for key, value in res.items():
      if key == 'results':
        for key2, value2 in value.items():
          if len(value2)==0:
            pass
          else:
            no1 = value2[0]
            sujet =no1.get("item")
            global uri
            uri = sujet.get("value")
            ID = uri.split('/')[4]
            get_infobox(donnees, ID)
  except JSONDecodeError:pass


donnees={}
working_dir = '/content/drive/My Drive/NER_BNF/'#@param {type:"string"}
xls_file = '/content/drive/My Drive/NER_BNF/bert_persos.xlsx' #@param {type:"string"}
colonne_entites = 'word'#@param {type:"string"}
with open(xls_file, "rb") as myfile:
  df = pd.read_excel(myfile)
  persos = df[colonne_entites].values.tolist()

for perso in persos:
  get_ID(perso)

nom_fichier = 'persos_complets.xlsx' #@param {type:"string"}
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel(''.join([working_dir, nom_fichier]))

In [ ]:
# @title Études statistiques

# Nombres de persos / siècle
#@markdown Ouvrir un fichier .XLSX pour effectuer les calculs statistiques
fichier_xlsx = '/content/drive/My Drive/NER_BNF/persos_complets.xlsx' #@param {type:"string"}
with open(fichier_xlsx, 'rb') as myFile:
  df3=pd.read_excel(myFile)
  liste_annees = df3["Naissance"].values.tolist()

def calcul_siecles(liste):
  liste_siecles=[]
  for annee in liste:
    annee_net = re.sub(r'[\']', "",str(annee))
    try:
      if (int(annee_net) <= 100):
        siecle = "1"
        liste_siecles.append(siecle)
      elif (int(annee_net) % 100 == 0):
        siecle=(annee_net // 100)
        liste_siecles.append(siecle)
      else:
        siecle=(int(annee_net) // 100 + 1)
        liste_siecles.append(siecle)
    except ValueError as valerr:
      pass

  occurrences_siecles = Counter(liste_siecles) ## Calcul du nombre d'occurences de chaque siècle
  df_siecles = pd.DataFrame.from_dict(occurrences_siecles, orient='index').reset_index()
  df_siecles.rename(columns={'index': 'Siècles'}, inplace=True)
  display(df_siecles.sort_values("Siècles"))

calcul_siecles(liste_annees)


# Statistiques / genre
col_list = df3["Genre"].values.tolist()
c = Counter(col_list) ## Affiche le nombre masculin / fémnin / None
prop = [(i, c[i] / len(col_list) * 100.0) for i in c]
df_genre = pd.DataFrame.from_dict(c, orient='index').reset_index()
df_genre.rename(columns={'index': 'Genre'}, inplace=True)
display(df_genre)